In [32]:
import numpy as np
# web scraping imports
from CNN import *
from Reuters import *
from SeekingAlpha import *
from CNBC import *

# install nltk, selenium, genism and bs4
# install chromedriver and add to path

# ntlk imports
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

# prettyprint
import pprint

# genism imports
from gensim import corpora,models
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS

stopwords = []  # user defined stopwords

def process_article(text):
# for single article (text string), tokenize and lemmatize data, return list of word stems
    sents = sent_tokenize(text)
    sents = '. '.join([s.strip().replace("\n", "") for s in sents])
    words = [word for word in sents.lower().split()
             if word not in STOPWORDS and word.isalnum() and word not in stopwords]
    wordnet_lemmatizer = WordNetLemmatizer()
    words = [wordnet_lemmatizer.lemmatize(i) for i in words]
    p_stemmer = PorterStemmer()
    words = [p_stemmer.stem(i) for i in words]
    return words


def get_corpus(text):
# for single article (text string), tokenize data and return corpus (list of lists)
    a_list = [process_article(text)]
    dictionary = corpora.Dictionary(a_list)
    return [dictionary.doc2bow(a) for a in a_list]


#https://radimrehurek.com/gensim/models/tfidfmodel.html
# models.TfidfModel: normalize=True by default,normalize document vectors to unit euclidean length.You can also inject your own function into normalize.
#slope=0.65 by default: Parameter required by pivoted document length normalization which determines the slope to which the old normalization can be tilted. This parameter only works when pivot is defined.
def tf_idf(corpus):
    average_count=[]
    for item in corpus:
        if len(item) != 0:
            average_count.append(np.mean(item,axis=0)[1])
        else:
            average_count.append(0)
    pivot = np.mean(average_count)    
    tfidf = models.TfidfModel(corpus,pivot=pivot)
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf


def get_lda(news_list, num_topics, num_passes):
# for list of articles ([date,text string] list), generate lda model
    texts = [process_article(a[1]) for a in news_list]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(a) for a in texts]
    
    average_count=[]
    for item in corpus:
        if len(item) != 0:
            average_count.append(np.mean(item,axis=0)[1])
        else:
            average_count.append(0)
    pivot = np.mean(average_count)
    
    tfidf = models.TfidfModel(corpus,pivot=pivot)
    corpus_tfidf = tfidf[corpus]
    lda = LdaModel(corpus_tfidf,  # list of lists containing tuples (word index, word freq)
                   id2word=dictionary,  # change nums back to words
                   num_topics=num_topics,  # need to set num topics
                   passes=num_passes)
    return lda,corpus_tfidf
 

def print_lda(lda, num_words=8):
# prints lda model coefficients, user can specify number of words to include for each topic
    pp = pprint.PrettyPrinter(indent=4)
    # create prettyprint obj, 8 words for each topic
    pp.pprint(lda.print_topics(num_words=8))


def get_topic(article_number,corpus_tfidf):
# for a single article (text string), given the order of the article on the loaded news list, returns list of relevant topics ordered by likelihood
    from operator import itemgetter
    #single_corpus = get_corpus(article) 
#     lda.get_document_topics(new_a[0],minimum_probability=0.05,per_word_topics=False)
    return sorted(lda.get_document_topics(corpus_tfidf[article_number],minimum_probability=0, per_word_topics=False),
                  key=itemgetter(1), reverse=True)



In [33]:
import os
import datetime

def get_news(ticker,days):
# retreives news from websites, may take a few minutes
    #get_cnn(ticker,days)
    #get_reuters(ticker,days)
    #get_seekingalpha(ticker,days)
    get_cnbc(ticker,days)

def get_date(filename,current_dir):
# for given filename and relative directory, return date (datetime format)
    try:
        date = datetime.datetime.strptime(filename[:19],'%Y-%m-%d_%H.%M.%S')
        return date
    except:
        pass
    
def get_file(filename,current_dir):
# for given filename and relative directory, return file content (text string)
    date = get_date(filename,current_dir)
    if date:
        with open(current_dir+"/"+filename,"r") as f:
            text = f.read()
        return text

def load_news(ticker,days=3):
# for given ticker, loads news from database and returns a list of [date,articles]
# default parameter: load articles from recent three days
    l = []
    for news_source in os.listdir("news/"+ticker):
        current_dir = "news/"+ticker+"/"+news_source
        for doc in os.listdir(current_dir):
            date = get_date(doc,current_dir) 
            if date and date>=(datetime.datetime.now()-datetime.timedelta(days)):
                l.append([date,get_file(doc,current_dir)])
    return l

In [34]:
def load_dict():
# load negative and positive dictionaries
    import os
    def get_dict_words(dict_dir):
        with open(dict_dir,'r') as f:
            words = []
            for line in f:
                words.append(line.replace('\n','').lower())
        return words
    
    neg = get_dict_words(os.getcwd() + '\\dictionaries\\negative.txt')
    pos = get_dict_words(os.getcwd() + '\\dictionaries\\positive.txt')
    return neg,pos

def emotion_analysis(text):
# input: text (string)
# output: negative and positive score
    neg,pos = load_dict()
    from nltk import word_tokenize
    results = dict()
    c1 = c2 = 0
    for word in word_tokenize(text):
        word = word.lower()
        if word in neg:
            c1 += 1
        if word in pos:
            c2 += 1
    n = len(word_tokenize(text))
    results['negative'] = c1/n
    results['positive'] = c2/n
    return results

def emotion_analyzer(text_list):
# input: list of texts (list)
# output: dataframe containing negative and positive scores of each text
    import pandas as pd
    df = pd.DataFrame(columns=['negative','positive'])
    count = 1
    for text in text_list:
        a = emotion_analysis(text[1])
        df.loc[count] = [a['negative'],a['positive']]
        count += 1
    return df

In [35]:
def get_lda_noun(nouns_list, num_topics, num_passes):
# input: list of nouns (list), number of topics (int), number of passes (int)
# output: lda model and corpus_tfidf
    texts = [process_article_noun(a) for a in nouns_list]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(a) for a in texts]
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lda = LdaModel(corpus_tfidf,  # list of lists containing tuples (word index, word freq)
                   id2word=dictionary,  # change nums back to words
                   num_topics=num_topics,  # need to set num topics
                   passes=num_passes)
    return lda,corpus_tfidf

def process_article_noun(text):
# for single article (text string), tokenize and lemmatize data, return list of word stems
# for topic analysis on nouns
    sents = sent_tokenize(text)
    sents = '. '.join([s.strip().replace("\n", "") for s in sents])
    words = [word for word in sents.lower().split()
             if word not in STOPWORDS and word.isalnum() and word not in stopwords]
    wordnet_lemmatizer = WordNetLemmatizer()
    words = [wordnet_lemmatizer.lemmatize(i) for i in words]
    #p_stemmer = PorterStemmer()
    #words = [p_stemmer.stem(i) for i in words]
    return words

def pick_nouns(doc,stopwords):
# input: doc (str) and stopwords (list)
# output: list of nouns (list)
    import nltk
    stopwords = [x.upper() for x in stopwords]
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sens = sent_detector.tokenize(doc)
    wpos = []
    for i in sens:
        word_list = nltk.word_tokenize(i)
        wpos.extend(nltk.pos_tag(word_list))
    result = []
    for i in wpos:
        if i[1] == 'NNP':
            if i[0].upper() not in stopwords:
                result.append(i[0])
    return result

def noun_lda(doc_list,stopwords,numTopic,numPass):
# input: list of news (list), stopwords(case does not matter) (list) , number of topics (int) and number of passes (int)
# output: lda model and corpus_tfidf
    nouns = []
    for i in text:
        nouns.append(' '.join(pick_nouns(i,stopwords)))
    lda,corpus_tfidf = get_lda_noun(nouns, numTopic, numPass)
    return lda,corpus_tfidf

In [36]:
def document_term_matrix(texts_list,num_topics,lda,stopwords,type= 'topic_terms'):
# input: list of texts (list), number of topics (int), lda model, list of stopwords (list), type of matrix generated (string)
# output: matrix (list)
    texts = [process_article_noun(a) for a in texts_list]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(a) for a in texts]
    
    nouns = []
    for i in texts_list:
        nouns.append(' '.join(pick_nouns(i,stopwords)))
    texts = [process_article_noun(a) for a in nouns]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(a) for a in texts]
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    
    if type == 'document_topics':
        dt = []
        for i in range(len(texts_list)):
            dt.append(get_topic(i,corpus_tfidf))
        return dt
    if type == 'topic_terms':
        tt = []
        for k,v in dictionary.items():
            pass
        word_map = dictionary.id2token
        for i in range(num_topics):
            tt.append(list(map(lambda x: (word_map[x[0]],x[1]),lda.get_topic_terms(i))))
        return tt
    
def exp_smooth(alpha,text_list):
# input: alpha in the exponential smoothing equation (double), list of texts (list)
# output: conslidated sentimental scores (tuple of doubles)
    neg = []
    pos = []
    for d in text:
        neg.append(emotion_analysis(d)['negative'])
        pos.append(emotion_analysis(d)['positive'])
    for i in range(len(text)):
        if i == 0:
            neg_s = neg[len(pos)-i-1]
            pos_s = pos[len(pos)-i-1]
        else:
            neg_s = alpha*neg[len(pos)-i-1] + (1-alpha)*neg_s
            pos_s = alpha*pos[len(pos)-i-1] + (1-alpha)*pos_s
    return neg_s,pos_s

In [37]:
get_news("fb",5)


Checking for win32 chromedriver:2.46 in cache
Driver found in C:\Users\Lora\.wdm\chromedriver\2.46\win32/chromedriver.exe
<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/amazons-advertising-sales-growth-slowed-dramatically-in-q1.html 2019-04-25 18:00:58.052172
<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/amazons-advertising-sales-growth-slowed-dramatically-in-q1.html 2019-04-25 18:00:58.232776
<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/reuters-america-update-1-new-york-investigating-facebook-over-storage-of-unauthorized-email-contacts.html 2019-04-25 18:00:58.643837
<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/reuters-america-update-3-canada-watchdog-to-seek-court-order-to-force-facebook-to-follow-privacy-laws.html 2019-04-25 18:00:59.062949
<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/new-york-attorney-general-to-probe-facebook.html 2019-04-25 17:00:59.240900
<class 'datetime.datetime'> https://www.cnbc.com/201

<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/reuters-america-update-1-new-york-investigating-facebook-over-storage-of-unauthorized-email-contacts.html 2019-04-25 18:01:18.571001
<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/amazons-advertising-sales-growth-slowed-dramatically-in-q1.html 2019-04-25 18:01:18.709163
<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/reuters-america-update-3-canada-watchdog-to-seek-court-order-to-force-facebook-to-follow-privacy-laws.html 2019-04-25 18:01:19.095844
<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/new-york-attorney-general-to-probe-facebook.html 2019-04-25 17:01:19.257619
<class 'datetime.datetime'> https://www.cnbc.com/2019/04/25/reuters-america-brief-canada-says-facebook-must-back-up-commitment-to-protect-canadians-personal-data.html 2019-04-25 17:01:19.582504
<class 'str'> https://www.cnbc.com/video/2019/04/25/facebook-bans-personality-quizzes-on-its-platforms.html --
<class 'str'> https:/

In [38]:
news = load_news("fb")

# print lda model
lda,corpus_tfidf = get_lda(news, 2, 10)
print_lda(lda)

# get topic for single document
# a = "'A version of this article first appeared in the Reliable Sources newsletter. You can sign up for free right here.   This is an incredibly difficult time for Alex Trebek, his family members, and the extended Jeopardy! family that spans the globe. Trebek showed tremendous courage by recording a candid video message to fans about his stage 4 pancreatic cancer diagnosis. He even managed to work in a joke about being under contract for three more years. Trebek was diagnosed earlier this week, and his video was released on Wednesday afternoon.  In a time that is all about what is keeping us apart, we got tough news today about someone who has always brought America together, literally for decades, CNN\'s Chris Cuomo said Wednesday night. I don\'t care what your race, color, creed, gender, or bank account level, you\'ve watched Jeopardy. Since 1984 Alex Trebek has been the smartest guy in our living rooms, teaching us, but more importantly, bringing us together. Trebek\'s show puts facts first, Cuomo said, and we need him, now mo"
# print(get_topic(a))


[   (   0,
        '0.006*"advertis" + 0.004*"amazon" + 0.003*"revenu" + 0.003*"growth" + '
        '0.003*"ad" + 0.002*"access" + 0.002*"dramat" + 0.002*"expect"'),
    (   1,
        '0.004*"canadian" + 0.004*"privaci" + 0.003*"therrien" + '
        '0.003*"nasdaq" + 0.003*"inform" + 0.003*"index" + 0.003*"govern" + '
        '0.003*"500"')]


In [39]:
emotion_analyzer(news)

,negative,positive
1,0.047414,0.006466
2,0.018801,0.011751
3,0.010135,0.003378
4,0.027356,0.006079
5,0.020525,0.005701
6,0.020525,0.005701
7,0.009634,0.013487
8,0.014085,0.005282
9,0.009634,0.013487
10,0.006270,0.022727


In [20]:
# news = load_news("aapl",10)

In [69]:
# text = []
# for i in news:
#     text.append(i[1])

In [105]:
# print_lda(noun_lda(text,sw,4,10)[0])

[   (   0,
        '0.014*"qualcomm" + 0.008*"netflix" + 0.008*"foxconn" + 0.008*"huawei" '
        '+ 0.008*"book" + 0.008*"display" + 0.008*"japan" + 0.007*"roku"'),
    (   1,
        '0.012*"acm" + 0.009*"dutch" + 0.008*"march" + 0.008*"qualcomm" + '
        '0.008*"huawei" + 0.007*"sec" + 0.007*"google" + 0.007*"app"'),
    (   2,
        '0.014*"qualcomm" + 0.008*"oled" + 0.007*"eps" + 0.007*"qcom" + '
        '0.007*"nasdaq" + 0.007*"chesler" + 0.006*"pro" + 0.006*"macbook"'),
    (   3,
        '0.010*"intel" + 0.009*"brussels" + 0.009*"thursday" + '
        '0.008*"qualcomms" + 0.008*"qualcomm" + 0.007*"project" + '
        '0.007*"titan" + 0.007*"spotifys"')]
